<a href="https://colab.research.google.com/github/AwkwaBear/EE-628/blob/main/sg13g2_LDO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Design and Optimization of Analog LDO Using sg13g2 PDK with Relational Graph Neural Network and Reinforcement Learning


|Name|
|:--:|
|Vikas Shukla |
|Zion McDowell|
|Jie Zhao|
|Anthony Gasbarro|


### Install dependencies

## First we follow a modified version of the [Dockerfile](https://github.com/iic-jku/IIC-OSIC-TOOLS/blob/main/_build/Dockerfile) build Script from the [IIC-OSIC-Tools Github Page](https://github.com/iic-jku/IIC-OSIC-TOOLS/tree/main)

This script is adapted one section at a time to the Google Colab environment's syntax

- sg13g2 [Process Development Kit]
- Ngspice [Circuit Simulation Tool]
- Conda [Python Development Virtual Environment]
- Pytorch [Machine Learning Framework]
- PyG [Graph Neural Network Library]
- Tabulate [Data Table Export Interface Library]

Note: that after the installation of these packages, **you may need to restart the runtime to use these newly installed packages**.

In [ ]:
import os

# install conda.
CONDA_PREFIX = os.environ.get('CONDA_PREFIX', None)
if not CONDA_PREFIX:
  !python -m pip install condacolab
  import condacolab
  condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:10
🔁 Restarting kernel...


## Install PDK and Simulation Tools

### Install IHP Open PDK

In [ ]:
# clone pdk
!git clone https://github.com/IHP-GmbH/IHP-Open-PDK.git

Cloning into 'IHP-Open-PDK'...
remote: Enumerating objects: 4885, done.
remote: Counting objects: 100% (1303/1303), done.
remote: Compressing objects: 100% (553/553), done.
remote: Total 4885 (delta 761), reused 1163 (delta 710), pack-reused 3582
Receiving objects: 100% (4885/4885), 315.09 MiB | 24.17 MiB/s, done.
Resolving deltas: 100% (2134/2134), done.
Updating files: 100% (3788/3788), done.


#### Install OpenVaf (Verilog-A Compiler)

In [ ]:
# install openvaf and compile models
!wget https://openva.fra1.cdn.digitaloceanspaces.com/openvaf_23_5_0_linux_amd64.tar.gz
!tar -xf openvaf_23_5_0_linux_amd64.tar.gz
!./openvaf ./IHP-Open-PDK/ihp-sg13g2/libs.tech/ngspice/openvaf/psp103_nqs.va
!cp ./IHP-Open-PDK/ihp-sg13g2/libs.tech/ngspice/openvaf/psp103_nqs.osdi .


--2024-02-23 09:53:24--  https://openva.fra1.cdn.digitaloceanspaces.com/openvaf_23_5_0_linux_amd64.tar.gz
Resolving openva.fra1.cdn.digitaloceanspaces.com (openva.fra1.cdn.digitaloceanspaces.com)... 104.18.13.192, 104.18.12.192, 2606:4700::6812:dc0, ...
Connecting to openva.fra1.cdn.digitaloceanspaces.com (openva.fra1.cdn.digitaloceanspaces.com)|104.18.13.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Cookie coming from openva.fra1.cdn.digitaloceanspaces.com attempted to set domain to digitaloceanspaces.com
Length: 54596847 (52M) [application/gzip]
Saving to: ‘openvaf_23_5_0_linux_amd64.tar.gz’

openvaf_23_5_0_linu 100%[===================>]  52.07M   109MB/s    in 0.5s    

2024-02-23 09:53:25 (109 MB/s) - ‘openvaf_23_5_0_linux_amd64.tar.gz’ saved [54596847/54596847]

Finished building psp103_nqs.va in 23.71s


#### Install Ngspice


In [ ]:
# clone ngspice
!apt install bison flex libx11-dev libx11-6 libxaw7-dev libreadline6-dev autoconf libtool automake -y
!git clone https://git.code.sf.net/p/ngspice/ngspice

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libreadline-dev' instead of 'libreadline6-dev'
autoconf is already the newest version (2.71-2).
autoconf set to manually installed.
automake is already the newest version (1:1.16.5-1.3).
automake set to manually installed.
libreadline-dev is already the newest version (8.1.2-1).
libreadline-dev set to manually installed.
libx11-6 is already the newest version (2:1.7.5-1ubuntu0.3).
libx11-6 set to manually installed.
libx11-dev is already the newest version (2:1.7.5-1ubuntu0.3).
libx11-dev set to manually installed.
The following additional packages will be installed:
  libfl-dev libfl2 libice-dev libsm-dev libxmu-dev libxmu-headers libxpm-dev libxt-dev
Suggested packages:
  bison-doc flex-doc libice-doc libsm-doc libtool-doc gcj-jdk libxaw-doc libxt-doc
The following NEW packages will be installed:
  bison flex libfl-dev libfl2 libice-dev libsm-dev libtool libxaw7-dev libxm

In [ ]:

# compile ngspice
!cd ngspice && ./compile_linux.sh

Running libtoolize
libtoolize: putting auxiliary files in '.'.
libtoolize: copying file './ltmain.sh'
libtoolize: putting macros in AC_CONFIG_MACRO_DIRS, 'm4'.
libtoolize: copying file 'm4/libtool.m4'
libtoolize: copying file 'm4/ltoptions.m4'
libtoolize: copying file 'm4/ltsugar.m4'
libtoolize: copying file 'm4/ltversion.m4'
libtoolize: copying file 'm4/lt~obsolete.m4'
Running aclocal 
Running autoheader
Running automake -Wall --copy --add-missing
configure.ac:46: installing './ar-lib'
configure.ac:42: installing './compile'
configure.ac:44: installing './config.guess'
configure.ac:44: installing './config.sub'
configure.ac:37: installing './install-sh'
configure.ac:37: installing './missing'
src/Makefile.am: installing './depcomp'
configure.ac: installing './ylwrap'
Running autoconf
configure.ac:967: warning: The macro `AC_TRY_LINK' is obsolete.
configure.ac:967: You should run autoupdate.
./lib/autoconf/general.m4:2920: AC_TRY_LINK is expanded from...
configure.ac:967: the top level

## Install Other Python Tools

In [ ]:
# Install Pytorch with cpu-only option.
!conda install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 cpuonly -c pytorch

# Check the version of Python (should be larger than 3.7) and Pytorch.
import torch
import sys
print("Python version")
print (sys.version)

!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"

# Install PyG to enable graph neural networks. We need to manually guide Colab to install the correct version of PyG based on which Pytorch version we have installed
# (https://colab.research.google.com/drive/17VFsRZeZa7rgtrQ39-RMiYIE0ePrSlgN?authuser=1#scrollTo=rFwf_0DrvXLv). In this case it is 1.13.1 with cpu-only option.
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cpu.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cpu.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.13.1+cpu.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.13.1+cpu.html
!pip install torch-geometric

# Check the version of PyG.
import torch_geometric
print(torch_geometric.__version__)

# Install Gymnasium for the RL environment.
!pip install gymnasium

# Check Gymnasium version.
import gymnasium as gym
print(gym.__version__)

# Install tabulate for showing simulation results nicely during the runtime later.
!pip install tabulate

# Similarly, check tabulate version.
import tabulate
print(tabulate.__version__)

Channels:
 - pytorch
 - conda-forge
Platform: linux-64
Solving environment: / - \ | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.1.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cpuonly
    - pytorch==1.13.1
    - torchaudio==0.13.1
    - torchvision==0.14.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |       2_kmp_llvm           6 KB  conda-forge
    blas-2.121                 |              mkl          14 KB  conda-forge
    blas-devel-3.9.0           |   21_linux64_mkl          14 KB  conda-forge
    ca-certificates-2024.2.2   |       hbcca054_0         152 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge

### Setting up SG13g2 circuit simulation environment


Create some directory to organize files.

Setting up Ngspice initialization for g13g2. Also do use 8 threads to speed up simulations.

In [ ]:
%%writefile .spiceinit
* ngspice initialization for SG13G2
* assert BSIM compatibility mode with "nf" vs. "W"
set ngbehavior=hsa
* "nomodcheck" speeds up loading time
set ng_nomodcheck
set num_threads=8

In [ ]:
%%writefile netlist.spice
* NMOS & PMOS VGS sweep

.lib "IHP-Open-PDK/ihp-sg13g2/libs.tech/ngspice/models/cornerMOSlv.lib" mos_tt

.param width=5u
X1 dn gn 0 0 sg13_lv_nmos w=width l=0.13u
vd    dn 0        dc   0.9
vg    gn 0        dc   0.9
X2 dp gp 0 0 sg13_lv_pmos w=width l=0.13u
ed    dp 0 dn 0 -1
eg    gp 0 gn 0 -1
.op
.option post brief nomod
.end

.control
pre_osdi ./psp103_nqs.osdi
save @n.x1.nsg13_lv_nmos[gm] @n.x1.nsg13_lv_nmos[ids] @n.x1.nsg13_lv_nmos[cgg] @n.x2.nsg13_lv_pmos[gm] @n.x2.nsg13_lv_pmos[ids] @n.x2.nsg13_lv_pmos[cgg]
dc vg 0 1.3 0.01
wrdata output.txt @n.x1.nsg13_lv_nmos[gm] @n.x1.nsg13_lv_nmos[ids] @n.x1.nsg13_lv_nmos[cgg] @n.x2.nsg13_lv_pmos[gm] @n.x2.nsg13_lv_pmos[ids] @n.x2.nsg13_lv_pmos[cgg]
.endc

In [ ]:
!ngspice -b netlist.spice

# Test Cells Below

In [ ]:
import math
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv("output.txt", delim_whitespace=True, header=None)
df.columns = ["vgs", "gmn", "", "idn", "", "cggn", "", "gmp", "", "idp", "", "cggp"]
df['gm_id_n'] = df['gmn']/df['idn']
df['fT_n'] = df['gmn']/df['cggn']/2/math.pi
df['gm_id_p'] = df['gmp']/df['idp']
df['fT_p'] = df['gmp']/df['cggp']/2/math.pi
df['fT_n_tot'] = df['gmn']/(df['cggn']+6e-15)/2/math.pi
df['fT_p_tot'] = df['gmp']/(df['cggp']+6e-15)/2/math.pi

In [ ]:

plt.semilogy(df['vgs'], df['idn'], label="N")
plt.semilogy(df['vgs'], df['idp'], label="P")
plt.xlabel("Vgs (V)")
plt.ylabel("Id (V)")
plt.legend(loc="lower right")
plt.grid()
plt.show()


In [ ]:
plt.plot(df['vgs'], df['gm_id_n'], label="N")
plt.plot(df['vgs'], df['gm_id_p'], label="P")
plt.xlabel(" Vgs (V)")
plt.ylabel("Gm/Id (S/A)")
plt.legend(loc="upper right")
plt.grid()
plt.show()

In [ ]:

plt.plot(df['vgs'], df['fT_n'], label="N")
plt.plot(df['vgs'], df['fT_p'], label="P")
plt.xlabel(" Vgs (V)")
plt.ylabel(" ft (Hz)")
plt.legend(loc="upper left")
plt.grid()
plt.show()

In [ ]:
#Strange kink in NMOS fT. Also, the peak value is too large for this technology (?), probably because I am not including cgsol and cgdol. On the other hand, why are these so large?
#cgsol           2.63377e-15           3.19575e-15
#cgdol           2.42174e-15           3.02492e-15

In [ ]:
#For the NMOS, the gate-source overlap capacitance is ~3.2fF/5um = 0.64 fF/um. This is about twice as large as I have seen in other 0.13um technologies.


plt.semilogx(df['idn'], df['fT_n_tot'], label="N")
plt.semilogx(df['idp'], df['fT_p_tot'], label="P")
plt.xlabel("Id (A)")
plt.ylabel("ft (Hz)")
plt.legend(loc="upper left")
plt.grid()
plt.show()

In [ ]:
plt.plot(df['vgs'], df['cggn'], label="N")
plt.plot(df['vgs'], df['cggp'], label="P")
plt.xlabel("Vgs (V)")
plt.ylabel("Cgg (F)")
plt.legend(loc="upper left")
plt.grid()
plt.show()


In [ ]:

plt.plot(df['vgs'], df['gmn'], label="N")
plt.plot(df['vgs'], df['gmp'], label="P")
plt.xlabel("Vgs (V)")
plt.ylabel("gm (F)")
plt.legend(loc="upper left")
plt.grid()
plt.show()